In [ ]:
using Pkg; Pkg.activate("..")

In [ ]:
using CreditRisk
using PyPlot

### Example showing necessity of mixture of factor shifts

In [ ]:
m = 1000
t = 2
pk = 0.05
lk = 1
q = 0.3

a₁ = 0.7
a₂ = 0.65
b₁ = √0.51
b₂ = √0.57

α₁ = 1 - m^(-1/3)
α₂ = 1 - (log(m))^(-1/2)

Gh₁ = (α₁*invnormcdf(1-pk) + α₂*b₁*invnormcdf(q))/a₁
Gh₂ = (α₁*invnormcdf(1-pk) + α₂*b₂*invnormcdf(q))/a₂

Gh₁,Gh₂

### Construct `Parameter`

In [ ]:
N = 1000  # m
C = 2     # 2
S = 2     # d
l = 0.3   # q

cmm = zeros(N,C)
p = 0.05
cmm[:,1] .= p
cmm[:,2] .= 1-p

ead = zeros(N) .+ 1
lgc = zeros(N,C)
lgc[:,1] .= 0
lgc[:,2] .= 1
cn = fill(2,N)

β = zeros(N,C)
β[1:2:N,:] .= [0.7 0]
β[2:2:N,:] .= [0 0.65]
denom = @. sqrt(1 - $sum((β).^2, dims=2))
denom = dropdims(denom; dims=2)

# H[n, c] = inverse_unit_Gaussian(∑ᵧ cmm[c(n), γ])
cum_cmm = cumsum(cmm, dims=2)
H = invnormcdf.(cum_cmm)

weights = zeros(N,C)
weights = ead ./ sum(ead)
weights = weights .* lgc

param = Parameter(N, C, S, l, cmm, ead, lgc, cn, β, H, denom, weights)

### plot the upper bound $F_x(z)$ of $log P(L>x | Z=z)$


Tail bound approximation 

$$
P(L>l | Z=z) ⫹ e^{F_x(z)}
$$
The upper bond given by 
$$
Fₓ(z) = -θ_m(z)l + \psi(θ_m(z),z)
\quad\quad
\psi(θ_m(z),z) = \sum_{n=1}^{N} \ln \sum_{c=1}^C p_n^c (z) e^{\theta w_n^c}
$$
where
$$
\psi(θ_m(z),z) = \underset{\theta}{argmin} \{ -\theta l + \psi(z,\theta) \}
$$

In GL2005, outer level twisting amounts to finding `z` that

$$
\underset{z}{max} P(L>x | Z=z) e^{-\frac{1}{2}z^Tz}
\quad
\rightarrow
\quad
\underset{z}{max} \{F_x(z) - \frac{1}{2} z^Tz\}
$$

Here we want to plot 
$$
\begin{align*}
obj_1 &= Fₓ(z) \\
obj_2 &= -log(-F_x(z) + \frac{1}{2} z^Tz)
\end{align*}
$$

In [ ]:
let (N, C, S, loss, cmm, ead, lgc, cn, β, H, denom, weights) = unpack(param)
    span = 10     # [0, range]
    step = 0.5


    phi0 = zeros(N, C+1)
    phi  = @view phi0[:,2:end]
    pnc = zeros(N, C)

    Ψ = init_Ψ()
    innerlevel = InnerLevelTwisting(N, C)

    let Fₓ(z) = begin
        @. phi = normcdf((H - $(β*z)) / denom)
        diff!(pnc, phi0; dims=2)
        twist!(innerlevel, pnc, weights, loss)
        θ = get_result(innerlevel)
        -θ*l + Ψ(θ, pnc, weights)
    end


    xs = -span:step:span
    ys = -span:step:span
    obj₁ = zeros(length(xs),length(ys))
    obj₂ = zeros(length(xs),length(ys))

    for i in 1:length(xs)
        for j in 1:length(ys)
            z = [xs[i]; ys[j]]
            obj₁[i,j] = Fₓ(z)
            obj₂[i,j] = -log10(-obj₁[i,j] + 0.5z'z)
        end
    end        

    fig = figure(figsize=(8,16))
    ax = fig[:add_subplot](2,1,1, projection = "3d")
#     ax[:view_init](elev=0, azim=90+180)
    sf=ax[:plot_surface](xs, ys, obj₁,alpha=0.8,cmap=ColorMap("summer"),antialiased=false)
    ct=ax[:contour](xs, ys, obj₁,extend3d=false,zdir="z",offset=minimum(obj₁),stride=0.2)
    ax[:set_xlabel]("z₁")
    ax[:set_ylabel]("z₂")
    ax[:set_zlabel]("Fₓ(z)")
    fig[:colorbar](sf,shrink=0.3)
        
    subplot(212)
    ax = fig[:add_subplot](2,1,2, projection = "3d")
    sf=ax[:plot_surface](xs, ys, obj₂,alpha=0.8,cmap=ColorMap("summer"),antialiased=false)
    ct=ax[:contour](xs, ys, obj₂,extend3d=false,zdir="z",offset=minimum(obj₁),stride=0.2)
    ax[:set_xlabel]("z₁")
    ax[:set_ylabel]("z₂")
    ax[:set_zlabel]("-log(-Fₓ(z) + 0.5 z^Tz)")
    fig[:colorbar](sf,shrink=0.3)

#     savefig("surf_obj1.pdf", format="pdf")
    end
end

In [ ]:
log10(10)

In [ ]:
# nz = 1000
# ne = 1000
# est = CreditRisk.bernoulli_mc(param,(nz,ne))
# est